In [ ]:
https://serversforhackers.com/c/so-you-got-yourself-a-loadbalancer
https://serversforhackers.com/c/load-balancing-with-haproxy
https://serversforhackers.com/c/using-nginx-as-a-load-balancer    

## 1 - Nginx/LVS/HAProxy Comparison
Nginx/LVS/HAProxy是目前使用最广泛的三种负载均衡软件，

一般对负载均衡的使用是随着网站规模的提升根据不同的阶段来使用不同的技术。具体的应用需求还得具体分析，如果是中小型的Web应用，比如日PV小于1000万，用Nginx就完全可以了；如果机器不少，可以用DNS轮询，LVS所耗费的机器还是比较多的；大型网站或重要的服务，且服务器比较多时，可以考虑用LVS。常见的硬件有比较昂贵的F5和Array等商用的负载均衡器。

目前关于网站架构一般比较合理流行的架构方案：Web前端采用Nginx/HAProxy+Keepalived作负载均衡器；后端采用mysql数据库一主多从和读写分离，采用LVS+Keepalived的架构。

## 2 - Nginx
### 2.1 - Nginx的优点
1、工作在网络的7层之上，可以针对http应用做一些分流的策略，比如针对域名、目录结构，它的**正则规则比HAProxy更为强大和灵活**，这也是它目前广泛流行的主要原因之一。
2、Nginx**对网络稳定性的依赖非常小**，理论上能ping通就就能进行负载功能，这个也是它的优势之一；相反LVS对网络稳定性依赖比较大.
3、Nginx安装和配置比较简单，测试起来比较方便，它基本能把错误用日志打印出来。LVS的配置、测试就要花比较长的时间了，LVS对网络依赖比较大。
3、可以承担高负载压力且稳定，在硬件不差的情况下一般能支撑几万次的并发量，负载度比LVS相对小些。
4、Nginx**可以通过端口检测到服务器内部的故障**，比如根据服务器处理网页返回的状态码、超时等等，并且会把返回错误的请求重新提交到另一个节点，不过其中缺点就是不支持url来检测。比如用户正在上传一个文件，而处理该上传的节点刚好在上传过程中出现故障，Nginx会把上传切到另一台服务器重新处理，而LVS就直接断掉了，如果是上传一个很大的文件或者很重要的文件的话，用户可能会因此而不满。
5、Nginx不仅仅是一款优秀的负载均衡器/反向代理软件，它同时也是功能强大的Web应用服务器。LNMP也是近几年非常流行的web架构，在高流量的环境中稳定性也很好。
6、Nginx现在作为Web反向加速缓存越来越成熟了，速度比传统的Squid服务器更快，可以考虑用其作为反向代理加速器。
7、Nginx可作为中层反向代理使用，这一层面Nginx基本上无对手，唯一可以对比Nginx的就只有lighttpd了，不过lighttpd目前还没有做到Nginx完全的功能，配置也不那么清晰易读，社区资料也远远没Nginx活跃。
8、Nginx也可作为静态网页和图片服务器，这方面的性能也无对手。还有Nginx社区非常活跃，第三方模块也很多。
9、Nginx同时还**能区分内外网**，如果是同时拥有内外网的节点，就相当于单机拥有了备份线路
10、

### 2.2 - Nginx的缺点
1、Nginx**仅能支持http、https和Email协议**，这样就在适用范围上面小些，这个是它的缺点。
2、对后端服务器的健康检查，只支持通过端口来检测，**不支持通过url来检测**。
3、**不支持Session的直接保持**，但能通过ip_hash来解决。

## 3 -LVS  linux virtual server
LVS：使用Linux内核集群实现一个高性能、高可用的负载均衡服务器，它具有很好的可伸缩性（Scalability)、可靠性（Reliability)和可管理性（Manageability)。
### 3.1 - LVS的优点
1、抗负载能力强、是工作在网络4层之上**仅作分发之用，没有流量的产生**，这个特点也决定了它在负载均衡软件里的性能最强的，对内存和cpu资源消耗比较低。
2、**配置性比较低**，这是一个缺点也是一个优点，因为没有可太多配置的东西，所以并不需要太多接触，大大减少了人为出错的几率。
3、工作稳定，因为其本身抗负载能力很强，自身有**完整的双机热备方案**，如LVS+Keepalived，不过我们在项目实施中用得最多的还是LVS/DR+Keepalived。
4、无流量，LVS只分发请求，而流量并不从它本身出去，这点保证了均衡器IO的性能不会收到大流量的影响。
5、**应用范围比较广**，因为LVS工作在4层，所以它几乎可以对所有应用做负载均衡，包括http、数据库、在线聊天室等等。
6、在使用上，一般最前端所采取的策略应是LVS，也就是DNS的指向应为LVS均衡器，LVS的优点令它非常适合做这个任务。重要的ip地址，最好交由LVS托管，比如数据库的 ip、webservice服务器的ip等等，这些ip地址随着时间推移，使用面会越来越大，如果更换ip则故障会接踵而至。所以将这些重要ip交给 LVS托管是最为稳妥的，这样做的唯一缺点是需要的VIP数量会比较多。

### 3.2 - LVS的缺点
1、软件本身**不支持正则表达式处理**
2、**不能做动静分离**；而现在许多网站在这方面都有较强的需求，这个是Nginx/HAProxy+Keepalived的优势所在。
3、如果是网站应用比较庞大的话，**LVS/DR+Keepalived实施起来比较复杂**，特别后面有Windows Server的机器的话，如果实施及配置还有维护过程就比较复杂了，相对而言，Nginx/HAProxy+Keepalived就简单多了。
4、LVS就比较依赖于网络环境，目前来看服务器在**同一网段内**并且LVS使用direct方式分流，效果较能得到保证。
5、另外注意，LVS需要向托管商至少**申请多一个ip来做Visual IP**，貌似是不能用本身的IP来做VIP的。要做好LVS管理员，确实得跟进学习很多有关网络通信方面的知识，就不再是一个HTTP那么简单了。
6、LVS中 ldirectd也能支持针对服务器内部的情况来监控，但**LVS的原理使其不能重发请求**。比如用户正在上传一个文件，而处理该上传的节点刚好在上传过程中出现故障，Nginx会把上传切到另一台服务器重新处理，而LVS就**直接断掉**了，如果是上传一个很大的文件或者很重要的文件的话，用户可能会因此而恼火。

### 3.3 - LVS DR(Direct Routing)模式的网络流程图：
<img src="images/lvs_dr.jpg" style="width:500px;height:600px;"/>


## 4 - HAProxy
### 4.1 - HAProxy的优点
1、支持Session的保持，
2、Cookie的引导；
3、同时支持通过获取指定的url来检测后端服务器的状态。
4、HAProxy跟LVS类似，本身就只是一款负载均衡软件；单纯从效率上来讲HAProxy会比Nginx有更出色的负载均衡速度，
5、在并发处理上也是优于Nginx的。
6、HAProxy支持TCP协议的负载均衡转发，可以对MySQL读进行负载均衡，对后端的MySQL节点进行检测和负载均衡。
7、HAProxy负载均衡策略非常多，HAProxy的负载均衡算法现在具体有如下8种：
① roundrobin，表示简单的轮询，这个不多说，这个是负载均衡基本都具备的；
② static-rr，表示根据权重，建议关注；
③ leastconn，表示最少连接者先处理，建议关注；
④ source，表示根据请求源IP，这个跟Nginx的IP_hash机制类似，我们用其作为解决session问题的一种方法，建议关注；
⑤ ri，表示根据请求的URI；
⑥ rl_param，表示根据请求的URl参数’balance url_param’ requires an URL parameter name；
⑦ hdr(name)，表示根据HTTP请求头来锁定每一次HTTP请求；
⑧ rdp-cookie(name)，表示根据据cookie(name)来锁定并哈希每一次TCP请求。
8、SSL termination and SSL pass-through(because of TCP mode)
### 4.1 - HAProxy的缺点
1、No **caching and compression** capabilities


## Sessions

Session information is often saved on a temporary location within a web server.A user may log in, creating a session on one web server. On a subsequent request, however, the load balancer may bounce that user to another web server which doesn't have that session information! To the user, it appears that they are no longer logged in.

There are three common fixes for this.
### 1 - Cookie-based Sessions

In this scenario, cookies are used to store the session information. The session data, such as the user ID, are not saved to the server or any other storage, but are instead within the browser's cookie.

This has a limitation of the available amount of data a cookie can store. It's also easy to make insecure unless done correctly - the cookie needs to be encrypted in a way that can't be unencrypted, even if the cookie is hijacked by a malicious user.

### 2- Sticky Sessions

Another solution is use "sticky sessions", also called "session affinity". This will track which server a client was routed to and always route their request to the same web server on subsequent requests.

This let's the web server keep its default behavior of saving the session locally, leaving it up to the load balancer to get a client back to that server.

This is nice if you have a legacy application where changing the way cookies are handled is too difficult. However, this can skew the sharing of work load around your web servers.

You can see how to accomplish that in [HAProxy under the Load Balancing Algorithms section](https://serversforhackers.com/load-balancing-with-haproxy) here and [Nginx, under the available algorithms](https://serversforhackers.com/using-nginx-as-a-load-balancer) here.

#### HAProxy Session Affinity:
backend nodes
    # Other options above omitted for brevity
    cookie SRV_ID prefix
    server web01 127.0.0.1:9000 cookie check
    server web02 127.0.0.1:9001 cookie check
    server web03 127.0.0.1:9002 cookie check

#### Nginx Session Affinity:
upstream app_example {
    ip_hash;                 
    server 127.0.0.1:9000;
    server 127.0.0.1:9001;
    server 127.0.0.1:9002;
}

### 3- Central Session Storage

The second fix for this is to use a shared session storage mechanism.

Session storage is typically centralized within an in-memory stores such as Redis or Memcached. Persistent stores such as a database are also commonly used.

Since session data does not necessarily need the guaranteed persistence of a database, but may be heavily used, an in-memory data store's efficiency may be preferred.

In any case, using a cache (Redis, Memcached) for session-storage lets all the web servers connect to a central session store, growing your infrastructure a bit, but letting your work load be truly distributed across all web nodes.

Sample framework configuration to use Memcached:
```
<?php

# Session configuration for a Laravel application.
# Avoid using "file" in a load-balanced environment
return [
    /*
    |--------------------------------------------------------------------------
    | Default Session Driver
    |--------------------------------------------------------------------------
    |
    | This option controls the default session "driver" that will be used on
    | requests. By default, we will use the lightweight native driver but
    | you may specify any of the other wonderful drivers provided here.
    |
    | Supported: "file", "cookie", "database", "apc",
    |            "memcached", "redis", "array"
    |
    */
    'driver' => 'memcached',
];
```

https://www.digitalocean.com/community/tutorials/an-introduction-to-haproxy-and-load-balancing-concepts
https://www.digitalocean.com/community/tutorials/how-to-use-floating-ips-on-digitalocean#how-to-implement-an-ha-setup

## Haproxy High Level
HAProxy, which stands for High Availability Proxy, is a popular open source software **TCP/HTTP Load Balancer** and proxying solution which can be run on Linux, Solaris, and FreeBSD. Its most common use is to improve the performance and reliability of a server environment by distributing the workload across multiple servers (e.g. web, application, database). It is used in many high-profile environments, including: GitHub, Imgur, Instagram, and Twitter.
### HAProxy Terminology
#### Access Control List (ACL)

In relation to load balancing, ACLs are used to test some condition and perform an action (e.g. select a server, or block a request) based on the test result. Use of ACLs allows flexible network traffic forwarding based on a variety of factors like pattern-matching and the number of connections to a backend, for example.

Example of an ACL:
```
acl url_blog path_beg /blog
```
This ACL is matched if the path of a user's request begins with /blog. This would match a request of http://yourdomain.com/blog/blog-entry-1, for example.

For a detailed guide on ACL usage, check out the [HAProxy Configuration Manual](http://cbonte.github.io/haproxy-dconv/configuration-1.4.html#7).
#### Backend

A backend is a set of servers that receives forwarded requests. Backends are defined in the backend section of the HAProxy configuration. In its most basic form, a backend can be defined by:

* which load balance algorithm to use
* a list of servers and ports

A backend can contain one or many servers in it--generally speaking, adding more servers to your backend will increase your potential load capacity by spreading the load over multiple servers. Increase reliability is also achieved through this manner, in case some of your backend servers become unavailable.

Here is an example of a two backend configuration, web-backend and blog-backend with two web servers in each, listening on port 80:
```
backend web-backend
   balance roundrobin
   server web1 web1.yourdomain.com:80 check
   server web2 web2.yourdomain.com:80 check

backend blog-backend
   balance roundrobin
   mode http
   server blog1 blog1.yourdomain.com:80 check
   server blog1 blog1.yourdomain.com:80 check
```
**balance roundrobin** line specifies the load balancing algorithm, which is detailed in the [Load Balancing Algorithms section](https://www.digitalocean.com/community/tutorials/an-introduction-to-haproxy-and-load-balancing-concepts#load-balancing-algorithms).

**mode http** specifies that layer 7 proxying will be used, which is explained in [Types of Load Balancing](https://www.digitalocean.com/community/tutorials/an-introduction-to-haproxy-and-load-balancing-concepts#types-of-load-balancing) section.

The **check** option at the end of the server directives specifies that health checks should be performed on those backend servers.

#### Frontend

A frontend defines how requests should be forwarded to backends. Frontends are defined in the frontend section of the HAProxy configuration. Their definitions are composed of the following components:

* a set of IP addresses and a port (e.g. 10.1.1.7:80, *:443, etc.)
* ACLs
* use_backend rules, which define which backends to use depending on which ACL conditions are matched, and/or a default_backend rule that handles every other case

A frontend can be configured to various types of network traffic, as explained in the next section.

### Types of Load Balancing
Now that we have an understanding of the basic components that are used in load balancing, let's get into the basic types of load balancing.

#### No Load Balancing

A simple web application environment with no load balancing might look like the following:
<img src="images/web_server.png" />
In this example, the user connects directly to your web server, at yourdomain.com and there is no load balancing. If your single web server goes down, the user will no longer be able to access your web server. Additionally, if many users are trying to access your server simultaneously and it is unable to handle the load, they may have a slow experience or they may not be able to connect at all.

#### Layer 4 Load Balancing

The simplest way to load balance network traffic to multiple servers is to use layer 4 (transport layer) load balancing. Load balancing this way will forward user traffic based on IP range and port (i.e. if a request comes in for http://yourdomain.com/anything, the traffic will be forwarded to the backend that handles all the requests for yourdomain.com on port 80). 

<img src="layer_4_load_balancing" />

The user accesses the load balancer, which forwards the user's request to the web-backend group of backend servers. Whichever backend server is selected will respond directly to the user's request. Generally, all of the servers in the web-backend should be serving identical content--otherwise the user might receive inconsistent content. Note that both web servers connect to the same database server.

#### Layer 7 Load Balancing

Another, more complex way to load balance network traffic is to use layer 7 (application layer) load balancing. Using layer 7 allows the load balancer to forward requests to different backend servers based on the content of the user's request. This mode of load balancing allows you to run multiple web application servers under the same domain and port.
<img src="layer_7_load_balancing" />
In this example, if a user requests yourdomain.com/blog, they are forwarded to the blog backend, which is a set of servers that run a blog application. Other requests are forwarded to web-backend, which might be running another application. Both backends use the same database server, in this example.

A snippet of the example frontend configuration would look like this:
```
frontend http
  bind *:80
  mode http

  acl url_blog path_beg /blog
  use_backend blog-backend if url_blog

  default_backend web-backend
```
This configures a frontend named http, which handles all incoming traffic on port 80.

`acl url_blog path_beg /blog` matches a request if the path of the user's request begins with /blog.

`use_backend blog-backend if url_blog` uses the ACL to proxy the traffic to blog-backend.

`default_backend web-backend` specifies that all other traffic will be forwarded to web-backend.

### Load Balancing Algorithms
The load balancing algorithm that is used determines which server, in a backend, will be selected when load balancing. HAProxy offers several options for algorithms. In addition to the load balancing algorithm, servers can be assigned a weight parameter to manipulate how frequently the server is selected, compared to other servers.

Because HAProxy provides so many load balancing algorithms, we will only describe a few of them here. See the [HAProxy Configuration Manual](http://cbonte.github.io/haproxy-dconv/configuration-1.4.html#4.2-balance) for a complete list of algorithms.

A few of the commonly used algorithms are as follows:

**roundrobin**

Round Robin selects servers in turns. This is the default algorithm.

**leastconn**

Selects the server with the least number of connections--it is recommended for longer sessions. Servers in the same backend are also rotated in a round-robin fashion.

**source**

This selects which server to use based on a hash of the source IP i.e. your user's IP address. This is one method to ensure that a user will connect to the same server.

### Sticky Sessions
Some applications require that a user continues to connect to the same backend server. This persistence is achieved through sticky sessions, using the **appsession** parameter in the backend that requires it.

### Health Check
HAProxy uses health checks to determine if a backend server is available to process requests. This avoids having to manually remove a server from the backend if it becomes unavailable. **The default health check is to try to establish a TCP connection to the server i.e. it checks if the backend server is listening on the configured IP address and port.**

If a server fails a health check, and therefore is unable to serve requests, it is automatically disabled in the backend i.e. traffic will not be forwarded to it until it becomes healthy again. If all servers in a backend fail, the service will become unavailable until at least one of those backend servers becomes healthy again.

**For certain types of backends, like database servers in certain situations, the default health check is insufficient to determine whether a server is still healthy.**

### Other Solutions
If you feel like HAProxy might be too complex for your needs, the following solutions may be a better fit:

- Linux Virtual Servers (LVS) - A simple, fast layer 4 load balancer included in many Linux distributions
- Nginx - A fast and reliable web server that can also be used for proxy and load-balancing purposes. Nginx is often used in conjunction with HAProxy for its **caching and compression** capabilities

### High Availability
The layer 4 and 7 load balancing setups described before both use a load balancer to direct traffic to one of many backend servers. However, your load balancer is a single point of failure in these setups; if it goes down or gets overwhelmed with requests, it can cause high latency or downtime for your service.

A high availability (HA) setup is an infrastructure without a single point of failure. It prevents a single server failure from being a downtime event by adding redundancy to every layer of your architecture. A load balancer facilitates redundancy for the backend layer (web/app servers), but for a true high availability setup, you need to have **redundant load balancers** as well.

Here is a diagram of a basic high availability setup:
<img src="images/ha.gif" />
In this example, you have multiple load balancers (one active and one or more passive) behind a static IP address that can be remapped from one server to another. When a user accesses your website, the request goes through the external IP address to the active load balancer. If that load balancer fails, your failover mechanism will detect it and automatically reassign the IP address to one of the passive servers. There are a number of different ways to implement an active/passive HA setup. To learn more, read this section of How To Use Floating IPs.

## How To Implement an HA Setup

### Corosync and Pacemaker

Corosync and Pacemaker provide a cluster software package that can be used to create an effective HA setup. Corosync provides a messaging layer, which enables servers to communicate as a cluster, while Pacemaker provides the ability to control how the cluster behaves. This tutorial will demonstrate how you can use Corosync and Pacemaker with a Floating IP to create an active/passive HA server https://www.digitalocean.com/community/tutorials/how-to-create-a-high-availability-setup-with-corosync-pacemaker-and-floating-ips-on-ubuntu-14-04

### Keepalived

Keepalived is a service that can monitor servers or processes in order to implement high availability on your infrastructure. This guide uses the keepalived daemon to monitor two web servers. The secondary server will take over the web traffic if the primary server experiences issues by automatically claiming ownership of a shared 
https://www.digitalocean.com/community/tutorials/how-to-set-up-highly-available-web-servers-with-keepalived-and-floating-ips-on-ubuntu-14-04



## HAProxy SSL Support

### SSL Termination and Pass-through
[Using SSL Certificates with HAProxy](https://serversforhackers.com/c/using-ssl-certificates-with-haproxy)
If your application makes use of SSL certificates, then some decisions need to be made about how to use them with a load balancer.

A simple setup of one server usually sees a client's SSL connection being decrypted by the server receiving the request. Because a load balancer sits between a client and one or more servers, where the SSL connection is decrypted becomes a concern.

There are two main strategies.

- **SSL Termination** is the practice of terminating/decrypting an SSL connection at the load balancer, and sending unencrypted connections to the backend servers. This means the load balancer is responsible for decrypting an SSL connection - a slow and CPU intensive process relative to accepting non-SSL requests.
    - In this scenario, the load balancer alleviates the web servers of the extra CPU cycles needed to decrypt SSL traffic. 
    - It also gives the load balancer the opportunity to append the X-Forwarded-* headers to the request before passing it onward.
    -The downside of SSL Termination is that the traffic between the load balancers and the web servers is not encrypted. This leaves the application open to possible man-in-the-middle attacks.However, this is a risk usually mitigated by the fact that the load balancers are often within the same infrastructure (data center) as the web servers. Someone would have to get access to traffic between the load balancers and web servers by being within the data-centers internal network (possible, but less likely).
    - Amazon AWS load balancers also give you the option of generating a (self-signed) SSL for use between the load balancer and the web servers, giving you a secure connection all around. This, of course, means more CPU power being used, but if you need the extra security due to the nature of your application, this is an great option.

- This is the opposite of *SSL Pass-Through*, which sends SSL connections directly to the proxied servers. With SSL-Pass-Through, the SSL connection is terminated at each proxied server, distributing the CPU load across those servers. However, you lose the ability to add or edit HTTP headers, as the connection is simply routed through the load balancer to the proxied servers. This means your application servers will lose the ability to get the **X-Forwarded-* headers**, which may include the client's IP address, port and scheme used.
    - You also often lose some extra functionality that load-balancing proxies can provide, such as DDoS protection. 
    - SSL Pass-thru is only supported by load balancers that can balance traffic at the TCP level rather than HTTP level, since the traffic is not decrypted at the load balancer and is therefore not inspected to see what kind of traffic it is. That rules out Nginx for SSL pass-thru, but HAProxy will happily accomplish this for you

Which strategy you choose is up to you and your application needs. SSL Termination is the most typical I've seen, but pass-thru is likely more secure.

- There is a combination of the two strategies, where SSL connections are terminated at the load balancer, adjusted as needed, and then proxied off to the backend servers as a new SSL connection. This may provide the best of both security and ability to send the client's information. The trade off is more CPU power being used all-around, and a little more complexity in configuration.

### HAProxy with SSL Termination
We'll cover the most typical use case first - SSL Termination. As stated, we need to have the load balancer handle the SSL connection. This means having the SSL Certificate live on the load balancer server.

> Keep in mind that for a production SSL Certificate (not a self-signed one), you won't need to generate or sign a certificate yourself - you'll just need to create a Certificate Signing Request (csr) and pass that to whomever you purchase a certificate from.

#### Create Certifcate
```
$ sudo mkdir /etc/ssl/xip.io
$ sudo openssl genrsa -out /etc/ssl/xip.io/xip.io.key 1024
$ sudo openssl req -new -key /etc/ssl/xip.io/xip.io.key \
                   -out /etc/ssl/xip.io/xip.io.csr
> Country Name (2 letter code) [AU]:US
> State or Province Name (full name) [Some-State]:Connecticut
> Locality Name (eg, city) []:New Haven
> Organization Name (eg, company) [Internet Widgits Pty Ltd]:SFH
> Organizational Unit Name (eg, section) []:
> Common Name (e.g. server FQDN or YOUR name) []:*.xip.io
> Email Address []:

> Please enter the following 'extra' attributes to be sent with your certificate request
> A challenge password []:
> An optional company name []:
$ sudo openssl x509 -req -days 365 -in /etc/ssl/xip.io/xip.io.csr \
                    -signkey /etc/ssl/xip.io/xip.io.key \
                    -out /etc/ssl/xip.io/xip.io.crt
```
This leaves us with a xip.io.csr, xip.io.key and xip.io.crt file.

#### create pem
A pem file is essentially just the certificate, the key and optionally certificate authorities concatenated into one file. In our example, we'll simply concatenate the certificate and key files together (in that order) to create a xip.io.pem file. This is HAProxy's preferred way to read an SSL certificate.
```
$ sudo cat /etc/ssl/xip.io/xip.io.crt /etc/ssl/xip.io/xip.io.key \
           | sudo tee /etc/ssl/xip.io/xip.io.pem
```

> When purchasing a real certificate, you won't necessarily get a concatenated "bundle" file. You may have to concatenate them yourself. However, many do provide a bundle file. If you do, it might not be a pem file, but instead be a bundle, cert, cert, key file or some similar name for the same concept. This Stack Overflow answer explains that nicely.

#### Frontend Config
We'll setup our application to accept both http and https connections. To terminate an SSL connection in HAProxy, we can now add a binding to the standard SSL port 443, and let HAProxy know where the SSL certificates are:
```
frontend localhost
    bind *:80
    bind *:443 ssl crt /etc/ssl/xip.io/xip.io.pem
    mode http
    default_backend nodes
```

#### Backend
In the above example, we're using the backend "nodes". The backend, luckily, doesn't really need to be configured in any particular way. In the previous edition on HAProxy, we had the backend like so:
```
backend nodes
    mode http
    balance roundrobin
    option forwardfor
    option httpchk HEAD / HTTP/1.1\r\nHost:localhost
    server web01 172.17.0.3:9000 check
    server web02 172.17.0.3:9001 check
    server web03 172.17.0.3:9002 check
    http-request set-header X-Forwarded-Port %[dst_port]
    http-request add-header X-Forwarded-Proto https if { ssl_fc }
```
Because the SSL connection is terminated at the Load Balancer, we're still sending regular HTTP requests to the backend servers. We don't need to change this configuration, as it works the same!

#### SSL Only

If you'd like the site to be SSL-only, you can add a redirect directive to the frontend configuration:
```
frontend localhost
    bind *:80
    bind *:443 ssl crt /etc/ssl/xip.io/xip.io.pem
    redirect scheme https if !{ ssl_fc }
    mode http
    default_backend nodes
```
Above, we added the redirect directive, which will redirect from "http" to "https" if the connection was not made with an SSL connection. More information on ssl_fc is [available here](http://cbonte.github.io/haproxy-dconv/configuration-1.5.html#7.3.4-ssl_fc).

### HAProxy with SSL Pass-Through

With SSL Pass-Through, we'll have our backend servers handle the SSL connection, rather than the load balancer.

The job of the load balancer then is simply to proxy a request off to its configured backend servers. Because the connection remains encrypted, HAProxy can't do anything with it other than redirect a request to another server.

In this setup, we need to use **TCP mode over HTTP mode** in both the frontend and backend configurations. HAProxy will treat the connection as just a stream of information to proxy to a server, rather than use its functions available for HTTP requests.

First, we'll tweak the frontend configuration:
```
frontend localhost
    bind *:80
    bind *:443
    option tcplog
    mode tcp
    default_backend nodes
```
This **still binds to both port 80 and port 443**, giving the opportunity to use both regular and SSL connections.

As mentioned, to pass a secure connection off to a backend server without encrypting it, we need to use TCP mode (**mode tcp**) instead. This also means we need to set the logging to tcp instead of the default http (option **tcplog**). Read more on [log formats](http://cbonte.github.io/haproxy-dconv/configuration-1.5.html#8.2) here to see the difference between tcplog and httplog.

Next, we need to tweak our backend configuration. Notably, we once again need to change this to TCP mode, and we remove some directives to reflect the loss of ability to edit/add HTTP headers:
```
backend nodes
    mode tcp
    balance roundrobin
    option ssl-hello-chk
    server web01 172.17.0.3:443 check
    server web02 172.17.0.4:443 check
```
As you can see, this is set to **mode tcp** - Both frontend and backend configurations need to be set to this mode.

We also **remove option forwardfor** and the **http-request options** - these can't be used in TCP mode, and we couldn't inject headers into a request that's encrypted anyway.

**For health checks, we can use ssl-hello-chk which checks the connection as well as its ability to handle SSL (SSLv3 specifically) connections.**

In this example, I have two fictitious server backend that accept SSL certificates. If you've read the edition SSL certificates, you can see how to integrate them with Apache or Nginx in order to create a web server backend, which handles SSL traffic. **With SSL Pass-Through, no SSL certificates need to be created or used within HAproxy.** The backend servers can handle SSL connections just as they would if there was only one server used in the stack without a load balancer.

### Automatic SSL passthrough and termination

You first have to try checking for **passthrough, using SNI**. If nothing matches with the SNI info, it goes to a backend that **redirects to an alternate frontend** bound to a different port **with certs**, and will then perform **SSL termination**.

[Web application name to backend mapping in HAProxy](http://www.haproxy.com/blog/web-application-name-to-backend-mapping-in-haproxy/)

```
global
   log 127.0.0.1 local0 debug
   chroot /var/lib/haproxy
   stats socket /var/lib/haproxy/admin.sock level admin
   pidfile /var/lib/haproxy/pid
   stats timeout 30s
   daemon
   tune.ssl.default-dh-param 2048
   ssl-server-verify none
   crt-base /etc/ssl/certs/

defaults
   log global
   option tcplog
   option http-server-close
   timeout connect 5000
   timeout client 50000
   timeout server 50000

frontend stats 
   bind *:9999
   mode http
   stats enable
   stats refresh 30s
   stats show-node
   stats auth admin:admin
   stats show-desc Super Awesome Stats Page! YAY!   
   stats uri /haproxy?stats

frontend http_front
   bind *:80
   default_backend http_back
   mode http
	# Placeholder with funny page. Do we need to redirect to pt 443?

frontend https_Main
   bind *:443
   mode tcp
   mode tcp
   tcp-request inspect-delay 5s
   tcp-request content accept if { req_ssl_hello_type 1 }
   use_backend %[req.ssl_sni,lower,map_dom(/etc/haproxy/sni2backend.map,nonSNIhttps_back)]
	# Check the mapping for sni headers. If SNI info not fount, go to nonSNIhttps_back->nonSNIhttps_front

frontend nonSNIhttps_front
   bind 127.0.0.1:9443 ssl crt test01.pem crt test02.pem 
   mode http
   option forwardfor
   use_backend %[req.hdr(host),lower,map_dom(/etc/haproxy/domain2backend.map,http_back)]
	# This is the fallback to look for certs. If the domain is not found in the main FrontEnd, look here

backend nonSNIhttps_back
   mode tcp
   server nonSNIhttps_front 127.0.0.1:9443 check check-ssl  

backend http_back
   mode http
   server localhost 127.0.0.1:9998 check

backend test01
   mode http
   server test01 172.20.0.141:80 check

backend test02
   mode http
   server test02 172.20.0.142:80 check

backend dev42
   mode tcp 
   server dev42 172.20.0.42:443 check
```

现在对网络负载均衡的使用是随着网站规模的提升根据不同的阶段来使用不同的技术：

第一阶段：利用Nginx或HAProxy进行单点的负载均衡，这一阶段服务器规模刚脱离开单服务器、单数据库的模式，需要一定的负载均衡，但是仍然规模较小没有专业的维护团队来进行维护，也没有需要进行大规模的网站部署。这样利用Nginx或HAproxy就是第一选择，此时这些东西上手快， 配置容易，在七层之上利用HTTP协议就可以。这时是第一选择。

第二阶段：随着网络服务进一步扩大，这时单点的Nginx已经不能满足，这时使用LVS或者商用Array就是首要选择，Nginx此时就作为LVS或者Array的节点来使用，具体LVS或Array的是选择是根据公司规模和预算来选择，Array的应用交付功能非常强大，本人在某项目中使用过，性价比也远高于F5，商用首选！但是一般来说这阶段相关人才跟不上业务的提升，所以购买商业负载均衡已经成为了必经之路。

第三阶段：这时网络服务已经成为主流产品，此时随着公司知名度也进一步扩展，相关人才的能力以及数量也随之提升，这时无论从开发适合自身产品的定制，以及降低成本来讲开源的LVS，已经成为首选，这时LVS会成为主流。
最终形成比较理想的基本架构为：Array/LVS — Nginx/Haproxy — Squid/Varnish — AppServer。

## Nginx配置
1.1）nginx.conf配置文件定义Server（不推荐）

1.2）单nginx.conf加载配置其它config文件

eg：include vhost/*.conf;

1.3）Nginx负载均衡：

#upstream www.qutouwang.org {   
#zone myapp1 64k; 
#server x.x.x.x:80 weight=1 max_fails=2 fail_timeout=30s slow_start=30s; 
#server x.x.x.x:80 weight=1 max_fails=2 fail_timeout=30s;   
#}  
#proxy_pass http://127.0.0.1:8080;

In [ ]:
Open Questions
* Distributed LB
* SSL, Intel QuickAssist
* Nginx supports SSL LB?
* HAProxy supports SSL LB?
* Tenant Certificate